<a href="https://colab.research.google.com/github/ffneiva/py-qui/blob/main/27-08-2022-automacao-de-processos-com-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Automação de Processos

### 1. Objetivos

- Analisar uma base de dados extensa
- Coletar dados da internet (web scraping)
- Enviar e-mails com imagens e tabelas geradas em python

##### Passo a passo

1. Importar e tratar base de dados
2. Pegar valor do dólar em tempo real
3. Calcular indicadores
4. Enviar e-mail para cada loja

### 2. Importar base de dados

In [ ]:
import pandas as pd

caminho = '/content/drive/MyDrive/0. Pasta/base_dados.xlsx'
vendas = pd.read_excel(caminho)
vendas

In [ ]:
produtos = pd.read_excel(caminho, sheet_name='Produtos')
produtos

In [ ]:
emails = pd.read_excel(caminho, sheet_name='E-mails')
emails

,Loja,E-mail
0,Avenida 44,ff.python1@gmail.com
1,Avenida 85,ff.python2@gmail.com
2,Avenida Anhanguera,ff.python5@gmail.com
3,Goiânia Shopping,ff.python6@gmail.com
4,Shopping Passeio das Águas,ff.python7@gmail.com
5,Shopping Flamboyant,ff.python8@gmail.com
6,Shopping Bougainville,ff.python9@gmail.com


##### Tratar base de dados

In [ ]:
vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Data        1048572 non-null  object
 1   Loja        1048573 non-null  object
 2   Produto     1048562 non-null  object
 3   Quantidade  1048575 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 32.0+ MB


In [ ]:
vendas['Data'] = pd.to_datetime(vendas['Data'], 
                                format='%Y-%m-%d %H:%M:%S',
                                errors='coerce')
vendas = vendas.dropna()
vendas

##### Calcular o lucro de cada produto

In [ ]:
produtos['Lucro'] = produtos['Valor Unitário (US$)'] - produtos['Custo Unitário (US$)']
produtos

##### Calcular os valores totais de cada venda

In [ ]:
venda = vendas.merge(produtos, how='outer', on='Produto')
venda['Lucro'] = venda['Lucro'] * venda['Quantidade']
venda['Valor Unitário (US$)'] = venda['Valor Unitário (US$)'] * venda['Quantidade']
venda['Custo Unitário (US$)'] = venda['Custo Unitário (US$)'] * venda['Quantidade']
venda = venda.rename(columns={'Valor Unitário (US$)': 'Valor Total',
                              'Custo Unitário (US$)': 'Custo Total'})
venda

### 3. Pegar cotação do dólar em tempo real

In [ ]:
import requests
from bs4 import BeautifulSoup

link = 'https://br.investing.com/currencies/usd-brl'
pagina = requests.get(link)
sopa = BeautifulSoup(pagina.content, 'html.parser')
resposta = sopa.find('span', class_='text-2xl')
dolar = float(resposta.getText().replace(',', '.'))
print(dolar)

5.0623


##### Atualizar valores da tabela

In [ ]:
venda[['Valor Total', 'Custo Total', 'Lucro']] = \
    venda[['Valor Total', 'Custo Total', 'Lucro']] * dolar
venda

,Data,Loja,Produto,Quantidade,Valor Total,Custo Total,Lucro
0,2017-05-27,Avenida 44,Short,2,122.328,33.23244,89.09556
1,2015-06-07,Avenida Anhanguera,Short,1,61.164,16.61622,44.54778
2,2018-03-23,Shopping Passeio das Águas,Short,5,305.820,83.08110,222.73890
3,2016-03-18,Avenida 44,Short,2,122.328,33.23244,89.09556
4,2013-09-02,Avenida 44,Short,5,305.820,83.08110,222.73890
...,...,...,...,...,...,...,...
1048508,2013-12-22,Shopping Flamboyant,Cueca,3,76.455,25.84179,50.61321
1048509,2017-07-16,Goiânia Shopping,Cueca,1,25.485,8.61393,16.87107
1048510,2017-10-26,Avenida 44,Cueca,5,127.425,43.06965,84.35535
1048511,2019-01-21,Avenida Anhanguera,Cueca,4,101.940,34.45572,67.48428


### 4. Calcular indicadores para cada loja

##### Agrupar base de dados por loja

In [ ]:
vendas_por_loja = venda.groupby('Loja').sum()
vendas_por_loja

,Quantidade,Valor Total,Custo Total,Lucro
Loja,,,,
Avenida 44,684989,8.058895e+07,2.378931e+07,5.679964e+07
Avenida 85,237822,2.614470e+07,7.864549e+06,1.828016e+07
Avenida Anhanguera,413549,4.764055e+07,1.431636e+07,3.332419e+07
Goiânia Shopping,463001,4.893406e+07,1.484391e+07,3.409015e+07
Shopping Bougainville,146685,1.592636e+07,4.790325e+06,1.113603e+07
Shopping Flamboyant,576562,6.628006e+07,2.033224e+07,4.594783e+07
Shopping Passeio das Águas,484902,5.212779e+07,1.586424e+07,3.626355e+07


##### Cálculo do Ticket Médio

In [ ]:
vendas_por_loja['Ticket Médio'] = vendas_por_loja['Lucro'] / vendas_por_loja['Quantidade']
vendas_por_loja = vendas_por_loja.rename(columns={'Lucro': 'Faturamento'})
vendas_por_loja

,Quantidade,Valor Total,Custo Total,Faturamento,Ticket Médio
Loja,,,,,
Avenida 44,684989,8.058895e+07,2.378931e+07,5.679964e+07,82.920513
Avenida 85,237822,2.614470e+07,7.864549e+06,1.828016e+07,76.864864
Avenida Anhanguera,413549,4.764055e+07,1.431636e+07,3.332419e+07,80.580990
Goiânia Shopping,463001,4.893406e+07,1.484391e+07,3.409015e+07,73.628681
Shopping Bougainville,146685,1.592636e+07,4.790325e+06,1.113603e+07,75.917999
Shopping Flamboyant,576562,6.628006e+07,2.033224e+07,4.594783e+07,79.692775
Shopping Passeio das Águas,484902,5.212779e+07,1.586424e+07,3.626355e+07,74.785326


##### Produto mais vendido

In [ ]:
produtos_mais_vendidos = []

for loja in vendas_por_loja.index:
    df_loja = vendas.loc[vendas['Loja']==loja]
    df_loja_produtos = df_loja[['Produto', 'Quantidade']].groupby('Produto').sum()
    produto = df_loja_produtos.loc[df_loja_produtos['Quantidade']==df_loja_produtos['Quantidade'].max()].index[0]
    produtos_mais_vendidos.append(produto)
vendas_por_loja['Produto Mais Vendido'] = produtos_mais_vendidos
vendas_por_loja

,Quantidade,Valor Total,Custo Total,Faturamento,Ticket Médio,Produto Mais Vendido
Loja,,,,,,
Avenida 44,684989,8.058895e+07,2.378931e+07,5.679964e+07,82.920513,Camiseta
Avenida 85,237822,2.614470e+07,7.864549e+06,1.828016e+07,76.864864,Camisa
Avenida Anhanguera,413549,4.764055e+07,1.431636e+07,3.332419e+07,80.580990,Camisa
Goiânia Shopping,463001,4.893406e+07,1.484391e+07,3.409015e+07,73.628681,Camisa
Shopping Bougainville,146685,1.592636e+07,4.790325e+06,1.113603e+07,75.917999,Camisa
Shopping Flamboyant,576562,6.628006e+07,2.033224e+07,4.594783e+07,79.692775,Camisa
Shopping Passeio das Águas,484902,5.212779e+07,1.586424e+07,3.626355e+07,74.785326,Camisa


##### Gráfico das vendas ao longo do tempo

In [ ]:
import matplotlib.pyplot as plt

graficos = []

for loja in vendas_por_loja.index:
    df_loja = venda.loc[venda['Loja']==loja]
    df_loja['Ano'] = df_loja['Data'].dt.year
    df_loja_por_ano = df_loja[['Ano', 'Lucro']].groupby('Ano').sum()

    fig = plt.figure(figsize=(12,7))
    plt.bar(x=df_loja_por_ano.index, height=df_loja_por_ano['Lucro'])
    plt.xlabel('Ano')
    plt.ylabel('Faturamento')
    plt.title(f'Faturamento ao longo do tempo da loja {loja}')
    graficos.append(fig)

vendas_por_loja['Gráficos'] = graficos
vendas_por_loja

##### Ranking de vendas

In [ ]:
ranking = vendas_por_loja[['Quantidade', 'Faturamento']].sort_values(by='Faturamento', ascending=False)
ranking

,Quantidade,Faturamento
Loja,,
Avenida 44,684989,5.679964e+07
Shopping Flamboyant,576562,4.594783e+07
Shopping Passeio das Águas,484902,3.626355e+07
Goiânia Shopping,463001,3.409015e+07
Avenida Anhanguera,413549,3.332419e+07
Avenida 85,237822,1.828016e+07
Shopping Bougainville,146685,1.113603e+07


##### Função que formata moeda

In [ ]:
def formata_moeda(numero):
    '''Retorna uma string no formato de moeda brasileira
    ao receber um número em formato float'''
    return str(f'R$ {numero:,.2f}').replace('.', ';').replace(',', '.'). replace(';', ',')

In [ ]:
ranking['Faturamento'] = [formata_moeda(valor) for valor in ranking['Faturamento']]
ranking

### 5. Envio dos e-mails

##### Instalar módulo para envio dos e-mails

In [ ]:
pip install redmail

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##### Enviar e-mails para cada loja

In [ ]:
from redmail import EmailSender
from tqdm import tqdm

remetente = 'polipython@gmail.com'
senha = 'jtvujhxejnkeuztd'
# Senha para MAC: kbsgxekzbbjnmpov

email = EmailSender(host='smtp.gmail.com',
                    port=587,
                    username=remetente,
                    password=senha)

corpo = '''
    <h1>Resultados da loja {{ loja }}</h1>

    <p>Resumo dos resultados para a loja <i>{{ loja }}</i> da companhia:</p>

    <ul>
        <li>Faturamento total de <b>{{ faturamento }}</b></li>
        <li><b>{{ quantidade }}</b> peças vendidas</li>
        <li>Ticket médio de <b>{{ ticket_medio }}</b></li>
        <li><b>{{ produto_mais_vendido }}</b> foi o produto mais vendido</li>
    </ul>

    <p>Segue o ranking de vendas de todas as lojas:</p>

    {{ ranking }}

    <p>E os resultados da loja {{ loja }} ao longo do período:</p>

    {{ grafico }}

    <p>Att.</p>
    '''

for i, loja in tqdm(enumerate(vendas_por_loja.index)):
    destinatario = emails['E-mail'][i]
    assunto = f'Resultados da loja {loja}'
    grafico = vendas_por_loja['Gráficos'][i]
    quantidade = vendas_por_loja['Quantidade'][i]
    faturamento = formata_moeda(vendas_por_loja['Faturamento'][i])
    ticket_medio = formata_moeda(vendas_por_loja['Ticket Médio'][i])
    produto_mais_vendido = vendas_por_loja['Produto Mais Vendido'][i]
    
    email.send(
        subject = assunto,
        sender = remetente,
        receivers=destinatario,
        html=corpo,
        body_params={'loja': loja,
                    'faturamento': faturamento,
                    'quantidade': quantidade,
                    'ticket_medio': ticket_medio,
                    'produto_mais_vendido': produto_mais_vendido},
        body_images={'grafico': grafico},
        body_tables={'ranking': ranking}
    )

7it [00:11,  1.62s/it]
